# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 2 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [17]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [18]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [19]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [20]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    
    demands = []
    for node in graph.nodes():
        demands.append(graph.node[node]['demand'])

    if sum(demands) != 0:
        raise nx.NetworkXUnfeasible("No Feasible Flow Satify the demand")
        
    G1 = graph.copy()
    G1.add_node('S')
    G1.add_node('T')
    for i in range(len(G1.node.keys())):
        if G1.node.keys()[i] != "S" and G1.node.keys()[i] != "T":
            if G1.node[G1.node.keys()[i]]['demand'] > 0:
                G1.add_edge(G1.node.keys()[i],"T")
                G1.edge[G1.node.keys()[i]]["T"]['capacity'] = G1.node[G1.node.keys()[i]]['demand']
            elif G1.node[G1.node.keys()[i]]['demand'] < 0:
                G1.add_edge("S",G1.node.keys()[i])
                G1.edge["S"][G1.node.keys()[i]]['capacity'] = - G1.node[G1.node.keys()[i]]['demand']
            else:
                pass
    
    flow_value, flow_dict = nx.maximum_flow(G1, 'S', 'T')
    
    maxflow = 0
    for (s1, s2) in G1.edges():
        if s1 == 'S':
            maxflow += flow_dict[s1][s2]
            
    demand = 0
    for node in G1.nodes():
        if node != 'S' and node != 'T':
            if G1.node[node]['demand'] > 0:
                demand += G1.node[node]['demand']

    if maxflow == demand: 
        flow_dict.pop('S',None)
        flow_dict.pop('T',None)
        for key in flow_dict:
            flow_dict[key].pop('S',None)
            flow_dict[key].pop('T',None)
    else:
        raise nx.NetworkXUnfeasible("No Feasible Flow Satify the demand")
        
    return flow_dict

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [21]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    key = flow.keys()
    Fin = []
    Fout = []
    Netflow = []
    fout = 0
    fin = 0
    for i in range(len(key)):
        for j in range(len(flow[key[i]].keys())):
            fout = fout + flow[key[i]][flow[key[i]].keys()[j]]
        Fout.append(fout)
        fout = 0
    
    for i in range(len(key)):
        for j in range(len(key)):
            for k in range(len(flow[key[j]].keys())):
                if key[i] == flow[key[j]].keys()[k]:
                    fin = fin + flow[key[j]][flow[key[j]].keys()[k]]            
        Fin.append(fin)
        fin = 0
    
    for i in range(len(key)):
        Netflow.append((Fin[i] - Fout[i]))
        
    dictionary = dict(zip(key, Netflow))
    return dictionary

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [22]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
